In [2]:
#For merging and rezising the Masks

import os, sys
from PIL import Image

#Change path here
path = r"/Users/deepitapai/Documents/TUM/MLMI Project/MontgomerySet_Original/"


def resizeImg(im):
    image_resized = im.resize((224,224), Image.ANTIALIAS)

    return image_resized

def mergeMasks(path,pic):

    try :
        img_left = Image.open(path+"ManualMask/leftMask/"+pic)
        img_right = Image.open(path+"ManualMask/rightMask/"+pic)

        width, height = img_left.size
        l_top, l_left, l_right, l_bottom = 0,0,int(width/2),height
        r_top, r_left, r_right, r_bottom = int(width/2)+1,0,width,height
        img_left = img_left.crop((l_top, l_left, l_right, l_bottom))
        img_right = img_right.crop((r_top, r_left, r_right, r_bottom))

        img_new = Image.new('L',(width, height),'black')
        img_new.paste(img_left,(l_top, l_left, l_right, l_bottom))
        img_new.paste(img_right,(r_top, r_left, r_right, r_bottom))
        
    except IOError:
        print ("cannot merge images for ", pic)
            
    return(img_new)


if __name__=="__main__":
    
    #Merge and Resize Masks
    if not os.path.exists(path+"/MergedMask/"):
        os.makedirs(path+"/MergedMask/")

    for pic in os.listdir(path+"ManualMask/leftMask"):
        if pic.endswith(".png"): 
            img_merged = mergeMasks(path,pic)
            img_resized = resizeImg(img_merged)  
            img_resized.save(path+"/MergedMask/"+pic)
    
    #Resize CXRs
    if not os.path.exists(path+"/ResizedCXR/"):
        os.makedirs(path+"/ResizedCXR/")

    for pic in os.listdir(path+"/CXR_png"):
        if pic.endswith(".png"): 
            img = Image.open(path+"/CXR_png/"+pic)
            img_resized = resizeImg(img)  
            img_resized.save(path+"/ResizedCXR/"+pic)
    
    print("Done")

Done


In [4]:
#Splitting the Montgomery Dataset in train, test, validation sets and writing the necessary text files

import os, sys
from PIL import Image
import random

#Change path here
path = r"/Users/deepitapai/Documents/TUM/MLMIProject/dataset_montgomery disease_labelled/"
#Change splits here
train_frac = 0.7
test_frac = 0.2
val_frac = 0.1 #rest for validation


def create_textfiles():

    train= open(path+"train.txt","w+")
    test= open(path+"test.txt","w+")
    val= open(path+"val.txt","w+")
    
    return train,test,val

def get_imgnames():
    path_CXR = path + "/ResizedCXR"
    imglist = []
    for img in os.listdir(path_CXR):  #getting all image names
        imglist.append(img)

    return imglist

def split(imglist):
    random.shuffle(imglist)
    sets = len(imglist)
    #print(sets)
    train_sets= int(sets*train_frac)
    test_sets= int(sets*test_frac+train_sets)
    #print(train_sets)
    #print(test_sets)
    train_list = imglist[:train_sets]
    test_list = imglist[train_sets:test_sets]
    val_list = imglist[test_sets:sets]
    
    return train_list,test_list,val_list
    
def write_textfiles(imglist,textfile):
    for img in imglist:
        textfile.write(path+"/ResizedCXR/"+img+" "+path+"/MergedMask_annotated/"+img+"\n")
    
    textfile.close() 
    
if __name__=="__main__":
    
    train,test,val = create_textfiles()
    imglist = get_imgnames()
    train_list,test_list,val_list = split(imglist)
    write_textfiles(train_list,train)
    write_textfiles(test_list,test)
    write_textfiles(val_list,val)
    
    print("Done")

Done
